In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
import torch.nn.init as init
from torch.autograd import Variable
import scipy.io as sio
import torch.utils.data as Data
import matplotlib.pyplot as plt

In [45]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(5, 64, kernel_size=17, padding=8)
        init.xavier_uniform_(self.conv1.weight)
        self.conv_bn1 = nn.BatchNorm1d(5)
        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(2496, 1)
        self.conv2 = nn.Conv1d(64, 64, kernel_size=17, padding=8)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=17, padding=8)
        self.conv_bn2 = nn.BatchNorm1d(64)
        self.conv4 = nn.Conv1d(128, 128, kernel_size=17, padding=8)
        self.conv5 = nn.Conv1d(128, 192, kernel_size=17, padding=8)
        self.conv_bn3 = nn.BatchNorm1d(128)
        self.conv_bn4 = nn.BatchNorm1d(192)
        self.conv6 = nn.Conv1d(192, 192, kernel_size=17, padding=8)

    def forward(self, x, hidden=None):
        x = self.conv_bn1(x)
        x = F.relu(self.conv1(x))
        x = self.dropout(x)
        
        x = self.conv_bn2(x)
        x = F.relu(self.conv2(x))
        x = self.dropout(x)
        x = F.max_pool1d(x, 2)
        
        x = self.conv_bn2(x)
        x = F.relu(self.conv2(x))
        x = self.dropout(x)
        
        x = self.conv_bn2(x)
        x = F.relu(self.conv3(x))
        x = self.dropout(x)
        
        x = self.conv4(x)
        x = F.max_pool1d(x, 2)
        x = self.conv_bn3(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        x = self.conv_bn3(x)
        x = F.relu(self.conv5(x))
        x = self.dropout(x)
        
        x = self.conv6(x)
        x = F.max_pool1d(x, 2)
        x = F.relu(self.conv_bn4(x))
        x = self.dropout(x)
# #         print(x)
        
#         print(x.size())
        x = x.view(-1, 2496)
#         print(x.size())
        x = self.fc1(x)
#         print(x.size())
        return F.sigmoid(x)

In [46]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # PyTorch v0.4.0
#model = Net().to(device)
#model = model.cuda()
model = Net()

In [47]:
summary(model, (5, 110))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm1d-1               [-1, 5, 110]              10
            Conv1d-2              [-1, 64, 110]           5,504
           Dropout-3              [-1, 64, 110]               0
       BatchNorm1d-4              [-1, 64, 110]             128
            Conv1d-5              [-1, 64, 110]          69,696
           Dropout-6              [-1, 64, 110]               0
       BatchNorm1d-7               [-1, 64, 55]             128
            Conv1d-8               [-1, 64, 55]          69,696
           Dropout-9               [-1, 64, 55]               0
      BatchNorm1d-10               [-1, 64, 55]             128
           Conv1d-11              [-1, 128, 55]         139,392
          Dropout-12              [-1, 128, 55]               0
           Conv1d-13              [-1, 128, 55]         278,656
      BatchNorm1d-14              [-1, 

/Users/xufanxiong/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [48]:
import numpy as np
def normali(input):
	'''
	This function aim to normalize X_train
	'''
	for i in range(input.shape[2]):
		tem = input[:, :, i]
		tem = tem/(np.max(tem) - np.min(tem))
		input[:, :, i] = tem
	return input

In [49]:
dataset = sio.loadmat('fft_train_1.mat')
Y_train = dataset['Y_train']
Y_train = (Y_train - 50) / 150
X_train = dataset['X_trainB']
X_train = normali(X_train)
X_train = X_train.transpose(0, 2, 1)
print(Y_train.shape)
print(X_train.shape)
x_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(Y_train).float()
torch_train = Data.TensorDataset(x_train, y_train)
train_loader = Data.DataLoader(dataset=torch_train,
                              batch_size=128,
                              shuffle=True,
                              num_workers=2)

(3055, 1)
(3055, 5, 110)


In [50]:
dataset = sio.loadmat('fft_test_1.mat')
Y_test = dataset['Y_test']
Y_test = (Y_test - 50) / 150
X_test = dataset['X_testB']
X_test = normali(X_test)
X_test = X_test.transpose(0, 2, 1)
print(Y_test.shape)
print(X_test.shape)
x_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(Y_test).float()
torch_test = Data.TensorDataset(x_test, y_test)
test_loader = Data.DataLoader(dataset=torch_test,
                              batch_size=148,
                              shuffle=True,
                              num_workers=2)

(148, 1)
(148, 5, 110)


In [51]:
from torch import optim
optimizer = optim.Adam(model.parameters(), 
                       lr=0.01, betas=(0.9, 0.999), 
                       eps=1e-6, weight_decay=1e-4)
criterion = nn.MSELoss()

In [52]:
for epoch in range(10):  
    
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        
        inputs, labels = data
        inputs, labels = Variable(inputs), Variable(labels)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()   
        
        optimizer.step()
        
        running_loss += loss.data[0]

        if i % 10 == 9: 
            print('[%d, %5d] loss: %.6f' \
                  % (epoch+1, i+1, running_loss / 3055))
            running_loss = 0.0
print('Finished Training')


/Users/xufanxiong/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/Users/xufanxiong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


[1,    10] loss: 0.000223
[1,    20] loss: 0.000152
[2,    10] loss: 0.000102
[2,    20] loss: 0.000091
[3,    10] loss: 0.000063
[3,    20] loss: 0.000064
[4,    10] loss: 0.000049
[4,    20] loss: 0.000049
[5,    10] loss: 0.000046
[5,    20] loss: 0.000040


Process Process-114:
Process Process-113:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/xufanxiong/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/xufanxiong/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/xufanxiong/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/xufanxiong/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/xufanxiong/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/Users/xufanxiong/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)


KeyboardInterrupt: 

In [27]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        #data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
#         if batch_idx % 5 == 0:
#             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                     epoch, batch_idx*len(data), len(train_loader.dataset),
#                     100.*batch_idx / len(train_loader), loss.data[0]))

In [32]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        #data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        output = model(data)
        criterion = nn.MSELoss()
        test_loss = criterion(output, target)
        
    print('testing loss: ', test_loss)
    print(output)

In [29]:
for epoch in range(10):
    train(epoch)
    test()

/Users/xufanxiong/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


testing loss:  tensor(0.1009, grad_fn=<MseLossBackward>)
testing loss:  tensor(0.0449, grad_fn=<MseLossBackward>)
testing loss:  tensor(0.0368, grad_fn=<MseLossBackward>)
testing loss:  tensor(0.0666, grad_fn=<MseLossBackward>)
testing loss:  tensor(0.0384, grad_fn=<MseLossBackward>)
testing loss:  tensor(0.0276, grad_fn=<MseLossBackward>)
testing loss:  tensor(0.0207, grad_fn=<MseLossBackward>)
testing loss:  tensor(0.0296, grad_fn=<MseLossBackward>)
testing loss:  tensor(0.0337, grad_fn=<MseLossBackward>)
testing loss:  tensor(0.0321, grad_fn=<MseLossBackward>)


In [33]:
for epoch in range(10):
    test()

testing loss:  tensor(0.0321, grad_fn=<MseLossBackward>)
tensor([[0.5041],
        [0.4384],
        [0.5406],
        [0.5844],
        [0.4485],
        [0.5504],
        [0.5819],
        [0.5851],
        [0.4551],
        [0.3369],
        [0.5691],
        [0.6827],
        [0.6198],
        [0.9360],
        [0.4865],
        [0.9569],
        [0.5674],
        [0.3869],
        [0.3639],
        [0.7441],
        [0.6767],
        [0.0341],
        [0.6715],
        [0.5417],
        [0.0553],
        [0.6871],
        [0.5257],
        [0.1060],
        [0.0840],
        [0.4413],
        [0.7790],
        [0.5630],
        [0.6925],
        [0.6173],
        [0.5216],
        [0.5430],
        [0.6495],
        [0.0860],
        [0.5139],
        [0.8592],
        [0.7556],
        [0.3349],
        [0.6844],
        [0.6168],
        [0.9307],
        [0.6607],
        [0.5421],
        [0.7238],
        [0.6382],
        [0.3700],
        [0.8235],
        [0.2999],
       

/Users/xufanxiong/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


testing loss:  tensor(0.0321, grad_fn=<MseLossBackward>)
tensor([[0.4893],
        [0.5630],
        [0.5421],
        [0.2400],
        [0.8621],
        [0.0962],
        [0.3953],
        [0.6607],
        [0.7490],
        [0.5473],
        [0.0840],
        [0.2280],
        [0.5139],
        [0.2774],
        [0.5744],
        [0.4485],
        [0.6208],
        [0.6971],
        [0.5147],
        [0.9569],
        [0.4390],
        [0.0700],
        [0.6198],
        [0.2513],
        [0.7441],
        [0.4759],
        [0.3491],
        [0.6215],
        [0.5390],
        [0.5998],
        [0.5786],
        [0.5159],
        [0.3438],
        [0.5175],
        [0.5216],
        [0.5295],
        [0.4384],
        [0.6871],
        [0.5178],
        [0.7420],
        [0.9307],
        [0.0612],
        [0.6072],
        [0.6173],
        [0.6177],
        [0.8235],
        [0.5406],
        [0.7238],
        [0.3497],
        [0.5709],
        [0.5851],
        [0.7861],
       